In [9]:
from __future__ import annotations
import numpy as np
from numpy.typing import ArrayLike
from typing import Dict, List, Tuple, Optional, Iterable
import pywt

In [ ]:
class EMGFeatureExtractor:
    """
    Compute EMG features for multi-channel monopolar EMG arrays.

    Parameters
    ----------
    fs : float
        Sampling rate in Hz.
    features : Iterable[str] | None
        Names of features to compute. If None, uses `default_features()`.
    zc_threshold : float
        Deadband threshold for zero-crossing/SSC/WAMP in signal units (e.g., mV).
    bands : List[Tuple[float, float]]
        List of frequency bands (Hz) for band energy/power features.
    ar_order : int
        Order of autoregressive coefficients.
    fractal : str
        'higuchi' or 'katz' for fractal dimension.
    window : Optional[int]
        If provided, features will be computed per window of `window` samples via
        `compute_over_windows`. For `compute`, the whole signal segment is used.
    step : Optional[int]
        Hop size for `compute_over_windows`. If None, equals `window` (no overlap).

    Notes
    -----
    Input shape is (n_samples, n_channels). Output shape is (n_features, n_channels)
    for `compute` or (n_windows, n_features, n_channels) for `compute_over_windows`.

    Available features (strings you can pass in `features`):
        Time-domain (TD):
            'IEMG', 'MAV', 'MAV1', 'MAV2', 'SSI', 'VAR', 'RMS', 'WL', 'DASDV',
            'ZC', 'SSC', 'WAMP', 'AAC', 'MAVSLP'
            'AR1'...'ARp' where p is `ar_order`
            'HjorthActivity', 'HjorthMobility', 'HjorthComplexity'
        Frequency-domain (FD):
            'MNF', 'MDF', 'PKF', 'SM1', 'SM2', 'SM3', 'TTP',
            'BandPower_[f1,f2]' for each band in `bands` (e.g., BandPower_[20.0,60.0])
            'FrequencyContent' (normalized spectral centroid = MNF / Nyquist)
        Nonlinear:
            'FractalDim' (Higuchi or Katz, set by `fractal`), 'SampleEntropy',
            'WaveletEntropy' (Shannon entropy of DWT subband energies, normalized)
    """

    # ------------------------- construction / config ------------------------- #
    def __init__(
        self,
        fs: float,
        features: Optional[Iterable[str]] = None,
        zc_threshold: float = 0.01,
        bands: Optional[List[Tuple[float, float]]] = None,
        ar_order: int = 4,
        fractal: str = 'higuchi',
        window: Optional[int] = None,
        step: Optional[int] = None,
    ):
        self.fs = float(fs)
        self.ar_order = int(ar_order)
        self.zc_threshold = float(zc_threshold)
        self.fractal = fractal.lower()
        self.window = window
        self.step = step if step is not None else window
        if bands is None:
            bands = [(20.0, 60.0), (60.0, 120.0), (120.0, 250.0)]  # typical sEMG bands
        self.bands = [(float(a), float(b)) for a, b in bands]

        if features is None:
            features = self.default_features()
        self.features = list(features)

        # Expand AR placeholders if needed
        self.features = self._expand_ar_features(self.features)
        # NEW: expand BandPower / BandEnergy placeholders into per-band names
        self.features = self._expand_band_features(self.features)

    @staticmethod
    def default_features() -> List[str]:
        return [
            # 16 commonly used (paper-influenced) TD/FD features
            'IEMG','MAV','MAV1','MAV2','SSI','VAR','RMS','WL','DASDV',
            'AR1','AR2','AR3','AR4',
            'HjorthActivity','HjorthMobility','HjorthComplexity',
            # Requested/extra
            'ZC','SSC','WAMP','AAC','MAVSLP',
            'MNF','MDF','PKF','SM1','SM2','SM3','TTP','FrequencyContent',
            'FractalDim','WaveletEntropy'
        ]

    @staticmethod
    def _expand_ar_features(names: Iterable[str]) -> List[str]:
        out: List[str] = []
        max_ar = 0
        for n in names:
            if n.upper() == 'AR':
                max_ar = max(max_ar, 4)  # default to 4 if unspecified
            elif n.upper().startswith('AR') and n[2:].isdigit():
                max_ar = max(max_ar, int(n[2:]))
                out.append(n)
            else:
                out.append(n)
        # if bare 'AR' specified, expand to AR1..ARmax
        if 'AR' in [n.upper() for n in names] or max_ar:
            out.extend([f'AR{k}' for k in range(1, max(1, max_ar)+1)])
            out = [n for i,n in enumerate(out) if n.upper() != 'AR' and n not in out[:i]]
        return out
    
    def _expand_band_features(self, names: Iterable[str]) -> List[str]:
        """
        Expand 'BandPower' / 'BandEnergy' placeholders into one feature
        per band, e.g. 'BandPower_[20.0,60.0]'.
        """
        out: List[str] = []
        for n in names:
            if n in ('BandPower', 'BandEnergy'):
                for a, b in self.bands:
                    out.append(f'BandPower_[{a:.1f},{b:.1f}]')
                    out.append(f'BandEnergy_[{a:.1f},{b:.1f}]')
            else:
                out.append(n)
        return out

    # ------------------------- public API ------------------------- #
    def compute(self, x: ArrayLike) -> np.ndarray:
        X = np.asarray(x, dtype=float)
        if X.ndim != 2:
            raise ValueError("Input must be 2D array of shape (n_samples, n_channels)")
        return self._compute_block(X)

    def compute_over_windows(self, x: ArrayLike, window: Optional[int]=None, step: Optional[int]=None) -> np.ndarray:
        X = np.asarray(x, dtype=float)
        if X.ndim != 2:
            raise ValueError("Input must be 2D array of shape (n_samples, n_channels)")
        W = int(window if window is not None else (self.window or X.shape[0]))
        H = int(step if step is not None else (self.step or W))
        if W <= 0 or H <= 0:
            raise ValueError("window and step must be positive")
        n = X.shape[0]
        starts = np.arange(0, max(1, n - W + 1), H, dtype=int)
        out = np.empty((len(starts), len(self.features), X.shape[1]))
        for i, s in enumerate(starts):
            out[i] = self._compute_block(X[s:s+W])
        return out

    # ------------------------- feature implementations ---------------------- #
    def _compute_block(self, X: np.ndarray) -> np.ndarray:
        n, C = X.shape
        feats = np.empty((len(self.features), C), dtype=float)

        # Pre-computations
        absX = np.abs(X)
        diffX = np.diff(X, axis=0)
        varX = np.var(X, axis=0, ddof=1) if n > 1 else np.zeros(C)
        rms = np.sqrt(np.mean(X**2, axis=0))
        wl = np.sum(np.abs(diffX), axis=0) if n > 1 else np.zeros(C)
        # PSD via real FFT
        freqs, psd = self._power_spectrum(X)

        # Hjorth
        dx = np.diff(X, axis=0)
        var_dx = np.var(dx, axis=0, ddof=1) if n > 1 else np.zeros(C)
        mob = np.sqrt(np.divide(var_dx, varX, out=np.zeros_like(varX), where=varX>0))
        ddx = np.diff(dx, axis=0)
        var_ddx = np.var(ddx, axis=0, ddof=1) if n > 2 else np.zeros(C)
        mob_dx = np.sqrt(np.divide(var_ddx, var_dx, out=np.zeros_like(var_dx), where=var_dx>0))
        comp = np.divide(mob_dx, mob, out=np.zeros_like(mob), where=mob>0)

        # AR coefficients
        ar = self._ar_coeffs(X, order=self.ar_order)  # shape (order, C)

        # Fractal, sample entropy
        if 'FractalDim' in self.features:
            if self.fractal == 'higuchi':
                frac = self._higuchi_fd(X)
            elif self.fractal == 'katz':
                frac = self._katz_fd(X)
            else:
                raise ValueError("fractal must be 'higuchi' or 'katz'")
        else:
            frac = None
        sampen = self._sample_entropy(X) if 'SampleEntropy' in self.features else None

        # Frequency helpers
        mnf = self._mean_frequency(freqs, psd)
        mdf = self._median_frequency(freqs, psd)
        pkf = freqs[np.argmax(psd, axis=0)] if psd.size else np.zeros(C)
        sm1, sm2, sm3, ttp = self._spectral_moments(freqs, psd)
        freq_content = self._frequency_content(freqs, psd)

        # Band powers/energies
        band_powers: Dict[str, np.ndarray] = {}
        for a, b in self.bands:
            val = self._band_power(freqs, psd, a, b)
            # Power in [a,b] Hz
            band_powers[f'BandPower_[{a:.1f},{b:.1f}]'] = val
            # "Energy" alias (same quantity, different name)
            band_powers[f'BandEnergy_[{a:.1f},{b:.1f}]'] = val

        # event-like counts (ZC/SSC/WAMP)
        zc = self._zero_crossings(X, self.zc_threshold)
        ssc = self._slope_sign_changes(X, self.zc_threshold)
        wamp = self._willison_amplitude(X, self.zc_threshold)

        # AAC, MAVSLP, Wavelet entropy
        aac = np.mean(np.abs(diffX), axis=0) if n > 1 else np.zeros(C)
        mavslp = self._mavslp(X)
        wavelet_entropy = self._wavelet_entropy(X)

        # Map feature names to arrays for assembly
        feature_map: Dict[str, np.ndarray] = {
            'IEMG': np.sum(absX, axis=0),
            'MAV': np.mean(absX, axis=0),
            'MAV1': self._mav1(X),
            'MAV2': self._mav2(X),
            'SSI': np.sum(X*X, axis=0),
            'VAR': varX,
            'RMS': rms,
            'WL': wl,
            'DASDV': np.sqrt(np.mean(diffX**2, axis=0)) if n > 1 else np.zeros(C),
            'ZC': zc,
            'SSC': ssc,
            'WAMP': wamp,
            'AAC': aac,
            'MAVSLP': mavslp,
            'HjorthActivity': np.var(X, axis=0),
            'HjorthMobility': mob,
            'HjorthComplexity': comp,
            'MNF': mnf,
            'MDF': mdf,
            'PKF': pkf,
            'SM1': sm1,
            'SM2': sm2,
            'SM3': sm3,
            'TTP': ttp,
            'FrequencyContent': freq_content,
            'FractalDim': frac if frac is not None else np.zeros(C),
            'WaveletEntropy': wavelet_entropy,
            'SampleEntropy': sampen if sampen is not None else np.zeros(C),
        }
        # ARk mapping
        for k in range(1, self.ar_order+1):
            feature_map[f'AR{k}'] = ar[k-1]
        # bands
        for key, val in band_powers.items():
            feature_map[key] = val

        # assemble in same order requested
        for i, name in enumerate(self.features):
            if name not in feature_map:
                raise KeyError(f"Unknown feature '{name}'.")
            feats[i] = feature_map[name]
        return feats

    # ------------------------- helpers: TD features -------------------------- #
    def _mav1(self, X: np.ndarray) -> np.ndarray:
        n = X.shape[0]
        w = np.ones(n)
        lo, hi = int(0.25*n), int(0.75*n)
        w[:lo] = 0.5; w[hi:] = 0.5
        return (np.abs(X)*w[:,None]).mean(axis=0)

    def _mav2(self, X: np.ndarray) -> np.ndarray:
        n = X.shape[0]
        w = np.empty(n)
        lo, hi = int(0.25*n), int(0.75*n)
        w[lo:hi] = 1.0
        if lo > 0:
            w[:lo] = 4*np.arange(lo)/n
        w[hi:] = 4*(np.arange(hi, n)-n)/n
        return (np.abs(X)*w[:,None]).mean(axis=0)

    def _zero_crossings(self, X: np.ndarray, thr: float) -> np.ndarray:
        x = X
        s = np.sign(x)
        s[s==0] = 1
        crosses = ((s[:-1,:] * s[1:,:]) < 0) & (np.abs(x[1:,:]-x[:-1,:]) >= thr)
        return crosses.sum(axis=0)

    def _slope_sign_changes(self, X: np.ndarray, thr: float) -> np.ndarray:
        x = X
        d1 = x[1:-1,:] - x[:-2,:]
        d2 = x[1:-1,:] - x[2:,:]
        cond = ((d1 * d2) > 0) & ((np.abs(d1) >= thr) | (np.abs(d2) >= thr))
        return cond.sum(axis=0)

    def _willison_amplitude(self, X: np.ndarray, thr: float) -> np.ndarray:
        d = np.abs(np.diff(X, axis=0))
        return (d >= thr).sum(axis=0)

    def _mavslp(self, X: np.ndarray) -> np.ndarray:
        n = X.shape[0]
        if n < 2:
            return np.zeros(X.shape[1])
        mav = np.mean(np.abs(X), axis=0)
        # Simple slope estimate across time using linear fit on |X|
        t = np.arange(n)
        A = np.vstack([t, np.ones_like(t)]).T
        # per-channel slope from OLS on |x|
        slopes = np.empty(X.shape[1])
        absX = np.abs(X)
        for c in range(X.shape[1]):
            beta, *_ = np.linalg.lstsq(A, absX[:,c], rcond=None)
            slopes[c] = beta[0]
        return slopes

    # ------------------------- helpers: AR coefficients ---------------------- #
    def _ar_coeffs(self, X: np.ndarray, order: int) -> np.ndarray:
        n, C = X.shape
        if n <= order:
            return np.zeros((order, C))
        # Solve least squares on lagged matrix per channel
        ar = np.empty((order, C))
        for c in range(C):
            x = X[:,c]
            Y = x[order:]
            Phi = np.column_stack([x[order-k-1:n-k-1] for k in range(order)])
            # Solve Y = Phi * a + e
            a, _, _, _ = np.linalg.lstsq(Phi, Y, rcond=None)
            ar[:,c] = a
        return ar

    # ------------------------- helpers: frequency domain -------------------- #
    def _power_spectrum(self, X: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        n = X.shape[0]
        # Remove DC to reduce leakage
        Xc = X - X.mean(axis=0, keepdims=True)
        # next power of 2 for speed
        nfft = int(2**np.ceil(np.log2(max(8, n))))
        F = np.fft.rfft(Xc, n=nfft, axis=0)
        freqs = np.fft.rfftfreq(nfft, d=1.0/self.fs)
        psd = (np.abs(F)**2) / (self.fs * n)
        return freqs, psd

    def _mean_frequency(self, f: np.ndarray, P: np.ndarray) -> np.ndarray:
        Psum = P.sum(axis=0)
        with np.errstate(invalid='ignore', divide='ignore'):
            mnf = (f[:,None] * P).sum(axis=0) / np.where(Psum>0, Psum, 1)
        mnf[~np.isfinite(mnf)] = 0.0
        return mnf

    def _median_frequency(self, f: np.ndarray, P: np.ndarray) -> np.ndarray:
        cumsum = np.cumsum(P, axis=0)
        half = (P.sum(axis=0) / 2.0)
        idx = np.array([np.searchsorted(cumsum[:,c], half[c]) for c in range(P.shape[1])])
        idx = np.clip(idx, 0, len(f)-1)
        return f[idx]

    def _spectral_moments(self, f: np.ndarray, P: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        m0 = P.sum(axis=0)
        m1 = (f[:,None] * P).sum(axis=0)
        m2 = ((f[:,None]**2) * P).sum(axis=0)
        m3 = ((f[:,None]**3) * P).sum(axis=0)
        with np.errstate(invalid='ignore', divide='ignore'):
            sm1 = m1 / np.where(m0>0, m0, 1)  # mean frequency
            sm2 = m2 / np.where(m0>0, m0, 1)
            sm3 = m3 / np.where(m0>0, m0, 1)
        ttp = m0  # total power
        sm1[~np.isfinite(sm1)] = 0
        sm2[~np.isfinite(sm2)] = 0
        sm3[~np.isfinite(sm3)] = 0
        return sm1, sm2, sm3, ttp

    def _band_power(self, f: np.ndarray, P: np.ndarray, f1: float, f2: float) -> np.ndarray:
        mask = (f >= f1) & (f < f2)
        if not np.any(mask):
            return np.zeros(P.shape[1])
        return P[mask,:].sum(axis=0)

    def _frequency_content(self, f: np.ndarray, P: np.ndarray) -> np.ndarray:
        """Normalized spectral centroid (0–1): MNF divided by Nyquist."""
        if P.size == 0:
            return np.zeros(P.shape[1] if P.ndim == 2 else 1)
        mnf = self._mean_frequency(f, P)
        nyq = max(1e-12, self.fs / 2.0)
        fc = mnf / nyq
        fc[~np.isfinite(fc)] = 0.0
        return np.clip(fc, 0.0, 1.0)

    # ------------------------- helpers: nonlinear --------------------------- #
    def _higuchi_fd(self, X: np.ndarray, kmax: Optional[int]=None) -> np.ndarray:
        # Vectorized-ish Higuchi FD per channel
        n, C = X.shape
        if kmax is None:
            kmax = max(5, min(8, n//10))
        ks = np.arange(1, kmax+1)
        lnks = np.log(ks)
        fd = np.empty(C)
        for c in range(C):
            x = X[:,c]
            Lk = []
            for k in ks:
                Lmk = []
                for m in range(k):
                    idx = np.arange(m, n, k)
                    if len(idx) < 2:
                        continue
                    diff = np.abs(np.diff(x[idx])).sum()
                    norm = (n - 1) / (len(idx)*k)
                    Lmk.append((diff * norm))
                Lk.append(np.mean(Lmk) if Lmk else 0.0)
            Lk = np.array(Lk)
            # Fit slope of log(Lk) vs log(1/k) -> FD
            y = np.log(Lk + 1e-12)
            A = np.vstack([np.log(1.0/ks), np.ones_like(ks, dtype=float)]).T
            beta, *_ = np.linalg.lstsq(A, y, rcond=None)
            fd[c] = beta[0]
        return fd

    def _katz_fd(self, X: np.ndarray) -> np.ndarray:
        n, C = X.shape
        t = np.arange(n)
        fd = np.empty(C)
        for c in range(C):
            x = X[:,c]
            L = np.sum(np.sqrt(1 + np.diff(x)**2))
            d = np.max(np.sqrt((t - t[0])**2 + (x - x[0])**2))
            if d == 0:
                fd[c] = 0.0
            else:
                fd[c] = np.log(n) / (np.log(n) + np.log(L/d))
        return fd

    def _sample_entropy(self, X: np.ndarray, m: int = 2, r: Optional[float] = None) -> np.ndarray:
        # Simple (not super-optimized) SampEn per channel
        n, C = X.shape
        if r is None:
            r = 0.2 * np.std(X, axis=0)  # per channel threshold
        se = np.empty(C)
        for c in range(C):
            x = X[:,c]
            rc = r[c] if np.ndim(r) else r
            if n <= m + 1:
                se[c] = 0.0
                continue
            def _phi(mm: int) -> float:
                N = n - mm + 1
                if N <= 1:
                    return 0.0
                Xmm = np.array([x[i:i+mm] for i in range(N)])
                count = 0
                for i in range(N-1):
                    d = np.max(np.abs(Xmm[i+1:] - Xmm[i]), axis=1)
                    count += np.sum(d <= rc)
                return count / (N*(N-1)/2)
            B = _phi(m)
            A = _phi(m+1)
            if A > 0 and B > 0:
                se[c] = -np.log(A/B)
            else:
                se[c] = 0.0
        return se

    def _wavelet_entropy(self, X: np.ndarray, wavelet: str = 'db4', level: Optional[int] = None) -> np.ndarray:
        """Wavelet entropy per channel from DWT subband energy distribution (0–1)."""
        n, C = X.shape
        H = np.empty(C)
        for c in range(C):
            x = X[:, c]
            try:
                w = pywt.Wavelet(wavelet)
                max_level = pywt.dwt_max_level(len(x), w.dec_len)
                L = level if level is not None else max(1, min(6, max_level))
                coeffs = pywt.wavedec(x, w, level=L)
                energies = np.array([np.sum(ci.astype(float)**2) for ci in coeffs], dtype=float)
                total = energies.sum()
                if total <= 0 or not np.isfinite(total):
                    H[c] = 0.0
                    continue
                p = np.clip(energies / total, 1e-12, 1.0)
                ent = -(p * np.log(p)).sum()
                H[c] = ent / np.log(len(p))
            except Exception:
                H[c] = 0.0
        return H

    # ------------------------- convenience utilities ----------------------- #
    def describe(self) -> List[str]:
        """Return the list of features (in order) that will be computed."""
        names: List[str] = []
        for name in self.features:
            if name in ('BandPower', 'BandEnergy'):
                for a, b in self.bands:
                    names.append(f'BandPower_[{a:.1f},{b:.1f}]')
                    names.append(f'BandEnergy_[{a:.1f},{b:.1f}]')
            else:
                names.append(name)
        return names

In [11]:
# sampling rate
fs = 2000  

# define bands for band energy (Hz)
bands = [(20.0, 60.0), (60.0, 120.0), (120.0, 250.0)]

# features we want per channel
feat_names = [
    "RMS",              # effort / amplitude
    "MNF",              # mean frequency
    "MDF",              # median frequency
    "PKF",              # peak frequency
    "IEMG",             # integrated EMG
    "VAR",              # variance

    # channel-based features you asked for:
    "ZC",               # zero crossing count
    "WL",               # waveform length
    "SSC",              # slope sign changes
    "WaveletEntropy",   # wavelet entropy
    "FractalDim",       # fractal dimension

    # band energy (one feature per band)
    "BandEnergy_[20.0,60.0]",
    "BandEnergy_[60.0,120.0]",
    "BandEnergy_[120.0,250.0]",
]

# create the extractor
feat_extractor = EMGFeatureExtractor(
    fs=fs,
    features=feat_names,
    zc_threshold=0.01,
    bands=bands,   # make sure EMGFeatureExtractor uses these bands
    window=None,   # whole trial at once
)


In [12]:
fs = 2000.0



In [4]:
if __name__ == "__main__":
    fs = 1000.0
    n_samples = 2000
    n_channels = 64
    # Fake EMG: band-limited noise (20-250 Hz) per channel
    rng = np.random.default_rng(0)
    t = np.arange(n_samples)/fs
    x = (rng.normal(size=(n_samples, n_channels))).astype(float)

    extractor = EMGFeatureExtractor(fs=fs)
    F = extractor.compute(x)  # shape (n_features, 64)
    #print("features x channels:", F.shape)
    #print(F)

    # Per 200 ms windows with 50% overlap
    W = extractor.compute_over_windows(x, window=int(0.2*fs), step=int(0.1*fs))
    #print("windows x features x channels:", W.shape)

In [ ]:
if __name__ == "__main__":
    fs = 1000.0
    n_samples = 2000
    n_channels = 64

    # Fake EMG: white noise (you could band-limit this later if you want)
    rng = np.random.default_rng(0)
    t = np.arange(n_samples) / fs
    x = rng.normal(size=(n_samples, n_channels)).astype(float)

    # Define the bands you care about
    bands = [(20.0, 60.0), (60.0, 120.0), (120.0, 250.0)]

    # Build a feature list that includes band power/energy
    band_feature_names = []
    for a, b in bands:
        band_feature_names.append(f'BandPower_[{a:.1f},{b:.1f}]')
        band_feature_names.append(f'BandEnergy_[{a:.1f},{b:.1f}]')

    features = [
        'RMS', 'MNF', 'MDF', 'TTP',
        *band_feature_names
    ]

    extractor = EMGFeatureExtractor(
        fs=fs,
        features=features,
        bands=bands,
    )

    # # Whole-segment features: shape (n_features, n_channels)
    # F = extractor.compute(x)
    # print("features x channels:", F.shape)
    # print("Feature names:", extractor.describe())
    # print("RMS of channel 0:", F[features.index('RMS'), 0])

    # # Check one band power / energy for channel 0
    # #bp_idx = features.index('BandPower_[20.0,60.0]')
    # be_idx = features.index('BandEnergy_[20.0,60.0]')
    # #print("BandPower 20–60 Hz, ch0:", F[bp_idx, 0])
    # print("BandEnergy 20–60 Hz, ch0:", F[be_idx, 0])

    # # Per 200 ms windows with 50% overlap
    # W = extractor.compute_over_windows(x, window=int(0.2*fs), step=int(0.1*fs))
    # print("windows x features x channels:", W.shape)
    # # Example: band power 20–60 Hz over windows for channel 0
    # #print("BandPower 20–60, ch0 over windows:", W[:, bp_idx, 0])


features x channels: (10, 64)
Feature names: ['RMS', 'MNF', 'MDF', 'TTP', 'BandPower_[20.0,60.0]', 'BandEnergy_[20.0,60.0]', 'BandPower_[60.0,120.0]', 'BandEnergy_[60.0,120.0]', 'BandPower_[120.0,250.0]', 'BandEnergy_[120.0,250.0]']
RMS of channel 0: 0.9794174347451813
BandEnergy 20–60 Hz, ch0: 0.08099135778714665
windows x features x channels: (19, 10, 64)
